In [3]:
!pip install sentence_transformers numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 7.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 103.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 125.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 3.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 59.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 130.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 115.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9

In [1]:
from batched.utils import AsyncMemoryCache

In [2]:
cache = AsyncMemoryCache()

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import batched

class SentenceEmbedder:
   def __init__(self, model_name='mixedbread-ai/mxbai-embed-large-v1'):
      self.model = SentenceTransformer(model_name)

   @batched.aio.dynamically(cache=cache)
   def embed_sentences(self, sentences: list[str]) -> list[np.ndarray]:
      # Convert sentences to embeddings
      return self.model.encode(sentences)

# Create an instance of SentenceEmbedder
embedder = SentenceEmbedder()

# Embed single sentences
single_sent = "This is a test sentence."
embedding = embedder.embed_sentences(single_sent)
#awaited_embedding = await embedder.embed_sentences.acall(single_sent)

# Embed a batch of 1000 sentences
batch_sentences = [f"This is test sentence number {i}." for i in range(1000)]
#batch_embeddings = embedder.embed_sentences(batch_sentences)
#awaited_batch_embeddings = await embedder.embed_sentences.acall(batch_sentences)

# Check the statistics
#stats = embedder.embed_sentences.stats

/home/pau/batched/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import tracemalloc
tracemalloc.start()
batch_sentences = [f"This is test sentence number 10000." for i in range(64)]
await embedder.embed_sentences(batch_sentences)
print(embedder.embed_sentences.stats)

TypeError: object function can't be used in 'await' expression

In [5]:
cache._cache.keys()

odict_keys(['This is test sentence number 10000.'])

In [6]:
cache._stats

BatchProcessorCacheStats(total_gets=128, total_sets=128, total_hits=0, hit_rate=0.0, utilization_rate=0.0001, total_pops=0.0, eviction_rate=0.0, total_get_time=0.00011348724365234375, total_set_time=0.00023484230041503906)

In [ ]:
# hit_rate, access_time/miss_penalty, eviction_rate (kinda related to the hit_rate, if high make cache larger), cache_utilization (if low make cache smaller), load_time (if high make cache smaller), latency_reduction (what is the ultimate cache gain?), mem_overhead